# kci 사이트에서 논문을 뽑아오는 크롤링 프로세스

In [9]:
from selenium import webdriver
import time
import re
import pandas as pd
import pickle
import numpy as np
from selenium.webdriver.common.alert import Alert
import requests

## 딱 각잡는데까지의 코드

In [10]:
pw = pickle.load( open('pw.pickle', 'rb') )

In [11]:
url = 'https://www.kci.go.kr/kciportal/main.kci'
try :
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(1)
    driver.set_window_size(5000, 5000)
    driver.find_element_by_css_selector('#loginType3').click()
    driver.find_element_by_css_selector('#userID').send_keys('jmanchung')
    time.sleep(1)
    driver.find_element_by_css_selector('#userPW').send_keys(pw)
    driver.find_element_by_css_selector('#mLogin > div > button > img').click()
    time.sleep(1)
    driver.find_element_by_css_selector('#gnb > li:nth-child(2) > a > img').click()
    time.sleep(1)
    driver.find_element_by_css_selector('#AKCFrm > div > div.head > table > tbody > tr:nth-child(2) > td:nth-child(1) > select').click()
    driver.find_element_by_css_selector('#AKCFrm > div > div.head > table > tbody > tr:nth-child(2) > td:nth-child(1) > select > option:nth-child(4)').click()
    driver.find_element_by_css_selector('#AKCFrm > div > div.head > table > tbody > tr:nth-child(2) > td:nth-child(2) > input').send_keys('한국증권학회')
    driver.find_element_by_css_selector('#AKCFrm > div > div.head > table > tbody > tr:nth-child(3) > td:nth-child(1) > select').click()
    driver.find_element_by_css_selector('#AKCFrm > div > div.head > table > tbody > tr:nth-child(3) > td:nth-child(1) > select > option:nth-child(5)').click()
    driver.find_element_by_css_selector('#AKCFrm > div > div.head > table > tbody > tr:nth-child(3) > td:nth-child(2) > input').send_keys('한국증권학회지')
    driver.find_element_by_css_selector('#AKCFrm > div > div.btn_wrap > div > input[type="image"]').click()
    time.sleep(1)
    driver.find_element_by_css_selector('#conLeft > div > h4:nth-child(5) > a').click()
    for k in range(17):
        driver.find_element_by_css_selector('#pubdt{}'.format(k)).click()
    driver.find_element_by_css_selector('#faceted_pubdt_list > li.chekReSearch.al_right > a > img').click()
    time.sleep(1)
except : pass

# 논문 다운로드 코드 시작 

In [8]:
for page in range(1,1000):
    try : 
        for i in range(1,60+1):
            references = []
            driver.execute_script('window.scrollTo(100,{});'.format(i*100))
            word = driver.find_element_by_css_selector('#poArtiSearList > table > tbody > tr:nth-child({}) > td:nth-child(2) > p'.format(i)).text
            information_list.append(word)
            driver.find_element_by_css_selector('#poArtiSearList > table > tbody > tr:nth-child({}) > td:nth-child(2) > p > label'.format(i)).click()
            time.sleep(1)
            re = requests.get(driver.current_url)
            dom = BeautifulSoup(re.text,'lxml')
            if len(dom.select('#sBody > div > div.articleRight > div.tools > p')) > 2 : 
                if driver.find_element_by_css_selector('#sBody > div > div.articleRight > div.tools > p:nth-child(2) > a').text\
                != '현재 페이지 인쇄':
                    driver.find_element_by_css_selector('#sBody > div > div.articleRight > div.tools > p:nth-child(2) > a').click()
            driver.back()
            time.sleep(1)
        driver.find_element_by_css_selector('#sBody > div.section > div > div > div > a:nth-child(5) > img').click()
        time.sleep(1)
    except Exception as e :
        print(e)
        print(e.args)
        break
    driver.close()

name 'information_list' is not defined
("name 'information_list' is not defined",)


## 논문 긁어오는 코드 시작

In [23]:
information_list = []
references_list = []

for page in range(1,1000):
    try : 
        for i in range(1,60+1):
            references = []
            driver.execute_script('window.scrollTo(100,{});'.format(i*100))
            word = driver.find_element_by_css_selector('#poArtiSearList > table > tbody > tr:nth-child({}) > td:nth-child(2) > p'.format(i)).text
            if '한국증권학회' in word and '한국증권학회지' in word : 
                information_list.append(word)
            else : break
            driver.find_element_by_css_selector('#poArtiSearList > table > tbody > tr:nth-child({}) > td:nth-child(2) > p > label'.format(i)).click()
            time.sleep(1)
            driver.execute_script('window.scrollTo(100,5000);')
            driver.find_element_by_css_selector('#refeList > img').click()
            length_ = int(','.join(re.findall('\d',driver.find_element_by_css_selector('#refeList').text)).replace(',',''))
            if length_ != 0 :
                for j in range(1,length_+1):
                    driver.execute_script('window.scrollTo(100,{})'.format(100*j))
                    try : 
                        try : 
                            references.append(driver.find_element_by_css_selector('#printArea > div.articleBody > div:nth-child(3) > div > p:nth-child({})'.format(j)).text)
                        except : 
                            references.append(driver.find_element_by_css_selector('#printArea > div.articleBody > div:nth-child(2) > div > p:nth-child({})'.format(j)).text)
                    except : 
                        references.append(driver.find_element_by_css_selector('#printArea > div.articleBody > div:nth-child(1) > div > p:nth-child({})'.format(j)).text)
                references_list.append(','.join(references))
            else : references_list.append('0')
            driver.back()
            time.sleep(1)
        driver.find_element_by_css_selector('#sBody > div.section > div > div > div > a:nth-child(5) > img').click()
        time.sleep(1)
    except Exception as e :
        print(e)
        print(e.args)
        break
driver.back()
driver.execute_script("location.reload();")

KeyboardInterrupt: 

In [24]:
len(information_list) , len(references_list)

(295, 295)

In [25]:
df = pd.DataFrame()
df['infor'] = information_list
df['refer'] = references_list
df.tail()

,infor,refer
290,Smart beta Strategy and Long-Short Factor Inve...,0
291,주택연금 제도의 재무적 가치\n안세륭 | 한국증권학회 | 한국증권학회지 | 47(5...,0
292,국민연금의 배당압력과 기업의 투자효율성\n노정희 | 한국증권학회 | 한국증권학회지 ...,0
293,선물·옵션 만기일의 주가조작 행위에 대한 실증분석\n양철원 | 한국증권학회 | 한국...,0
294,투자자는 지배주주가 이번 여름에 할 일을 알고 있다: 계열사 간 합병과 주주 간 이...,0


# more tuning process

In [304]:
testing_ls = [[j for i in [info.split('|') for info in information_list[idx].split('\n') ] for j in i] for idx in range(len(information_list))]
paper_name_list = [testing_ls[idx][0] for idx in range(len(testing_ls))]
name_list = [testing_ls[i][1:idx] for i in range(len(testing_ls)) for idx,val in enumerate(testing_ls[i]) if '증권' in val]
name_list = [','.join(name) for name in name_list]

In [309]:
driver.close()